In [ ]:
!pip install -q gradio pandas==2.2.2 matplotlib seaborn fpdf librosa transformers tts numba==0.57.0 git+https://github.com/openai/whisper.git
!pip install numpy==1.25


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.3/57.3 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 84.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 31.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 

In [1]:


import os
import gradio as gr
import librosa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from fpdf import FPDF
from sklearn.ensemble import IsolationForest
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import whisper
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
from pydub import AudioSegment


Found existing installation: numpy 1.24.3
Uninstalling numpy-1.24.3:
  Successfully uninstalled numpy-1.24.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 129.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pymc 5.21.2 requires numpy>=1.25.0, but you have numpy 1.24.3 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.24.3 which is incompatible.
blosc2 3.3.0 requires numpy>=1.26, but you have numpy 1.24.3 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.24.3 which is incompatible.
treescope 0.1.9 requires numpy>=1.25.2, but you have numpy 1.24.3 which is incompatible.
albumentations 2.0.5 requires numpy>=1.24.4, but you have numpy 1.24.3 which is incompatible.
albucore 0.0.23 requires numpy>=1.24.4, but you have numpy 1.24.3 which is incompatible.
jaxlib 0.5.1 req

ModuleNotFoundError: No module named 'gradio'

In [ ]:
model = whisper.load_model("base")
sia = SentimentIntensityAnalyzer()

history = []
df_all = pd.DataFrame()


In [ ]:
def trim_audio(audio_path, start_time, end_time):
    sound = AudioSegment.from_file(audio_path)
    trimmed = sound[start_time * 1000:end_time * 1000]
    trimmed_path = f"trimmed_{os.path.basename(audio_path)}"
    trimmed.export(trimmed_path, format="mp3")
    return trimmed_path

In [ ]:
def transcribe_audio(audio_path):
    result = model.transcribe(audio_path)
    return result['text']

In [ ]:
def extract_features(audio_path, start_sec, end_sec):
    trimmed_path = trim_audio(audio_path, start_sec, end_sec)
    text = transcribe_audio(trimmed_path)

    y, sr = librosa.load(trimmed_path)

    pause_count = np.sum(librosa.effects.split(y, top_db=20).shape[0])
    words = text.split()
    filler_words = ['uh', 'um', 'like', 'you know']
    filler_ratio = sum([words.count(w) for w in filler_words]) / max(len(words), 1)
    speech_rate = len(words) / (end_sec - start_sec)

    pitch = librosa.yin(y, fmin=50, fmax=300, sr=sr)
    pitch_std = np.std(pitch)
    energy = np.sum(y**2) / len(y)
    sentiment_score = sia.polarity_scores(text)['compound']

    return {
        "pause_count": pause_count,
        "filler_ratio": filler_ratio,
        "speech_rate": speech_rate,
        "pitch_std": pitch_std,
        "avg_energy": energy,
        "sentiment_score": sentiment_score,
        "transcript": text
    }


In [ ]:
def analyze_audio(file, start_sec, end_sec):
    global history, df_all
    features = extract_features(file, start_sec, end_sec)
    history.append(features)

    feature_cols = ["pause_count", "filler_ratio", "speech_rate", "pitch_std", "avg_energy", "sentiment_score"]
    X = np.array([[f[col] for col in feature_cols] for f in history])

    iso = IsolationForest(contamination=0.2, random_state=42)
    anomaly = iso.fit_predict(X)
    for i, val in enumerate(anomaly):
        history[i]["Anomaly Label"] = "At Risk" if val == -1 else "Normal"

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
    cluster = kmeans.fit_predict(X_scaled)
    for i, c in enumerate(cluster):
        history[i]["Cluster"] = int(c)

    df_all = pd.DataFrame(history)
    df_all.to_csv("analysis_results.csv", index=False)
    return features, "analysis_results.csv"

In [ ]:
def generate_plots():
    if df_all.empty:
        return None
    plt.figure(figsize=(6, 4))
    sns.scatterplot(data=df_all, x="filler_ratio", y="speech_rate", hue="Cluster", style="Anomaly Label")
    plt.title("Filler Ratio vs Speech Rate")
    plt.tight_layout()
    plt.savefig("plot1.png")
    return "plot1.png"

In [ ]:
def generate_pdf():
    if df_all.empty:
        return None
    latest = history[-1]
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    pdf.cell(200, 10, txt="Voice Cognition Analysis Report", ln=True, align='C')
    for k, v in latest.items():
        pdf.cell(200, 10, txt=f"{k}: {v}", ln=True)
    path = "report.pdf"
    pdf.output(path)
    return path

In [ ]:
with gr.Blocks() as app:
    with gr.Tab("🧪 Analyze Audio"):
        with gr.Row():
            audio_input = gr.Audio(type="filepath", label="Upload Audio File")
        with gr.Row():
            start = gr.Slider(0, 60, value=0, label="Start Time (sec)")
            end = gr.Slider(0, 60, value=15, label="End Time (sec)")
        with gr.Row():
            analyze_btn = gr.Button("Analyze")
        with gr.Row():
            output_json = gr.JSON()
            csv_file = gr.File(label="Download CSV")
        analyze_btn.click(fn=analyze_audio, inputs=[audio_input, start, end], outputs=[output_json, csv_file])

    with gr.Tab("📊 Comparison Plots"):
        plot_btn = gr.Button("Generate Plot")
        plot_img = gr.Image()
        plot_btn.click(fn=generate_plots, inputs=[], outputs=[plot_img])

    with gr.Tab("📄 Export Report"):
        pdf_btn = gr.Button("Export as PDF")
        pdf_file = gr.File()
        pdf_btn.click(fn=generate_pdf, inputs=[], outputs=[pdf_file])

app.launch()
